# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [38]:
import pandas as pd
import numpy as np
from scipy import stats

In [39]:
data = pd.io.stata.read_stata('C:/Users/joshu/Downloads/Data/EDA_racial_discrimination/data/us_job_market_discrimination.dta')

In [40]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [41]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


# 1. What test is appropriate for this problem? Does CLT apply?

Since the question is what the relationship is between race and callback rates, we will be doing a 2-sample Z-test. In order to see if the Central Limit Theorem applies, the sample size must be sufficiently large (n>30) and the proportions must be greater than 10. 

In [42]:
white = data[data.race == 'w']
black = data[data.race == 'b']

w = len(data[data.race == 'w'])
b = len(data[data.race == 'b'])

print('Number of White resumes: ', w)
print('Number of Black resumes: ', b)

w_cb = np.sum(white.call)
b_cb = np.sum(black.call)

print('')
print('Number of White callbacks: ', w_cb)
print('Number of Black callbacks: ', b_cb)

w_cb_p = w_cb / w
b_cb_p = b_cb / b

print('')
print('Proportion of White callbacks: ', w_cb_p)
print('Proportion of Black callbacks: ', b_cb_p)

Number of White resumes:  2435
Number of Black resumes:  2435

Number of White callbacks:  235.0
Number of Black callbacks:  157.0

Proportion of White callbacks:  0.09650924024640657
Proportion of Black callbacks:  0.06447638603696099


Since all conditions are met, the CLT applies for this case.

# 2. What are the null and alternate hypotheses?

The null hypothesis is that white and black sounding names have the same callback rates:

w_cb_p = b_cb_p

The alternate hypothesis is that white and black sounding names have different callback rates:
    
w_cb_p > b_cb_p

# 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [43]:
# Define the callback proportion difference
prop_diff = w_cb_p - b_cb_p

# Calculate the standard deviation of the test statistic
a = (w_cb + b_cb)/(w+b)
sigma = np.sqrt(a*(1-a)/w + a*(1-a)/b)
print('Standard Error: ', sigma)

# Generate Z-score
z = (w_cb_p - b_cb_p)/ sigma

# Assuming alpha = 0.05, generate the critical value
z_critical = stats.norm.ppf(1-0.5*0.05)
ME = z_critical * sigma
print('Margin of Error: ', ME)

# Generate P-value
p = (1 - stats.norm.cdf(z)) * 2
print('P-value:', p)

# Calculate the confidence intervals
CI = prop_diff + (np.array([-1,1])*ME)
print('Confidence Interval: ', CI)

Standard Error:  0.007796894036170457
Margin of Error:  0.015281631502169232
P-value: 3.983886837577444e-05
Confidence Interval:  [0.01675122 0.04731449]


Since the p-value is less than 0.05, we can reject the null hypothesis and conclude that white-sounding names have better callback rates than black-sounding names.

# 4. Write a story describing the statistical significance in the context or the original problem.

Based on a sample of over 4000 resumes, it has become clear that employers discriminate against black-sounding names. Whether this is intentionally or simply inherent, the analysis does show that employers significantly prefer white-sounding names over black-sounding names. By the numbers, people with white-sounding names have nearly a 50% greater call-back rate than people with black-sounding names.

# 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, this doesn't mean that race/name is the most important factor in callback success. This analysis was simply looking at how different races fare against each other. In order to assess the most important factor, it would be better to compare other factors such as gender or education. 